In [6]:
from pandas import DataFrame
import numpy as np
import os
import re

In [7]:
os.getcwd()

'/home/yuanzhaolin/VAE-AKF/jupyter'

In [12]:
#origin path list
# Path_list =[
#     'ckpt/winding/vrnn/vrnn_/2020-11-28_13-19-37',
#     'ckpt/winding/vaecl/vaecl_model.D=1/2020-11-29_15-45-38',
#     'ckpt/winding/vaecl/vaecl_model.D=5/2020-11-29_20-42-05',
#     'ckpt/winding/srnn/srnn_/2020-11-25_19-48-03',
#     'ckpt/west/vrnn/vrnn_/2020-11-28_21-59-55',
#     'ckpt/west/vaecl/vaecl_model.D=1/2020-11-29_16-23-01',
#     'ckpt/west/vaecl/vaecl_model.D=5/2020-11-29_22-12-03',
#     'ckpt/west/srnn/srnn_/2020-11-25_21-46-40',
#     'ckpt/cstr/vrnn/vrnn_/2020-11-28_08-51-15',
#     'ckpt/cstr/vaecl/vaecl_model.D=1/2020-11-29_14-11-52',
#     'ckpt/cstr/vaecl/vaecl_model.D=5/2020-11-29_17-21-47',
#     'ckpt/cstr/srnn/srnn_/2020-11-25_14-57-52'
# ]
def my_filter(path):
    log_path = os.path.join('..', path, 'log.out')
    s = open(log_path, 'r').readlines()
    log_path = os.path.join('..', path, 'test.out')
    test_s = open(log_path, 'r').readlines()
    if False:
        return False
    elif s[4] != 'history_length: 40\n' and s[5] != 'forward_length: 160\n':
        return False
    elif 'Error' in test_s[-2]:
        return False
    else:
        return True
    
def generating_dir(base_dir, root_dir):
    ret = []
    for file in os.listdir(os.path.join(root_dir, base_dir)):
        if file == 'tmp':
            continue
        path = os.path.join(base_dir, file)
        if os.path.isdir(os.path.join(root_dir, path)):
            ret = ret + generating_dir(path, root_dir)
        elif file == 'test.out' and os.path.exists(os.path.join(root_dir, base_dir, 'best.pth')):
            if my_filter(os.path.split(path)[0]):
                ret.append(os.path.split(path)[0])
            else:
                continue
    return ret
path_list = generating_dir('ckpt/', '../')


ckpt/west_con/west_con1/vaecl_model.D=1/2020-12-24_15-42-02
['ckpt/winding/srnn/srnn_model.D=1/2020-12-02_22-35-24', 'ckpt/winding/srnn/srnn_model.D=5/2020-12-03_01-23-11', 'ckpt/winding/srnn/srnn_model.D=3/2020-12-02_23-58-35', 'ckpt/winding/vaecl/vaecl_model.D=5/2020-12-01_12-06-05', 'ckpt/winding/vaecl/vaecl_model.D=5/2020-12-01_12-07-01', 'ckpt/winding/vaecl/vaecl_model.D=3/2020-12-01_11-15-02', 'ckpt/winding/vaecl/vaecl_model.D=3/2020-12-01_11-15-52', 'ckpt/winding/vaecl/vaecl_model.D=1/2020-12-01_10-43-25', 'ckpt/winding/vaecl/vaecl_model.D=1/2020-12-01_10-43-27', 'ckpt/winding/vrnn/vrnn_model.D=1/2020-12-02_19-30-27', 'ckpt/winding/vrnn/vrnn_model.D=3/2020-12-02_22-10-52', 'ckpt/winding/vaecl_net_decoder/vaecl_model.D=20/2020-12-01_19-11-48', 'ckpt/cstr/srnn/srnn_model.D=1/2020-12-02_13-04-20', 'ckpt/cstr/srnn/srnn_model.D=5/2020-12-02_19-07-53', 'ckpt/cstr/srnn/srnn_model.D=3/2020-12-02_15-44-21', 'ckpt/cstr/vaecl/vaecl_model.D=5/2020-12-01_09-30-04', 'ckpt/cstr/vaecl/vaecl_mod

In [9]:
n=len(path_list)
data=[]

data = list(set([path.split('/')[1] for path in path_list]))

#获取数据集/模型名称及个数
# for path in Path_list:
#     #result=re.findall(r'/(.*?)/',path)#查找两个'/'之间的字符
    
#     result=path.split('/')
#     print(result)
#     if result[1] in data:
#         print('repeat data')
#     else:
#         data.append(result[1])

#print(result) 
print(data)

['west', 'west_con', 'winding', 'cstr']


In [13]:
#单个frame生成，构建单个数据集path列表
temp_list=[]             #存放单个数据集的临时列表
dex=[]                   #行列表
col=[]                   #列列表
df=[]                    #DataFrame集合
#x=np.zeros((n_dex,n_col))#数值矩阵
for d in range(len(data)):
    print(d)
    for path in path_list:
        if path.split('/')[1] == data[d]:
            temp_list.append(path)
    print(temp_list)
    for path in temp_list:
        #pattern=re.compile(r'\/.*?/')
        result=path.split('/')
        #print(result)
        dex.append(result[len(result)-2]+'//'+result[len(result)-1])
    #print('dex', len(dex))
    n_dex=len(dex)#行数
    #print(dex)
    #assert len(dex) == len(temp_list)
    
    #单独开启第一个path文件获取列数以便于初始化数值矩阵
    f = open('../'+temp_list[0]+'/test.out','r')
    temp_data = f.readlines()
    f.close()
    for line in temp_data:
        if re.search('likelihood',line):
            #pattern = re.compile(r'-?[0-9]\d*\.\d*')   #查找数字(之查找包含小数点的)
            t_col = re.findall(r'(\w*\(?\w\)?\w*)=(-?[0-9]\d*\.\d*)',line)
            #print(t_col)
            for i in range(len(t_col)):
                if t_col[i][0]=='time':
                    pass
                else:
                    col.append(t_col[i][0])
    #print('col', len(col))
    n_col=len(col)           #列数
    x=np.zeros((n_dex,n_col))#数值矩阵
    for t in range(len(temp_list)):      #第t个临时列表中的路径
        #print('t=',t)
        f = open('../'+temp_list[t]+'/test.out','r')
        temp_data = f.readlines()
        f.close()
        #数值矩阵生成
        for line in temp_data:
            if re.search('likelihood',line):
                #print(line[11:len(line)])
                #print(len(line))
                pattern = re.compile(r'-?[0-9]\d*\.\d*')   #查找数字(之查找包含小数点的)
                result = pattern.findall(line)
                #数据对应填入该行
                t_x=re.findall(r'(\w*\(?\w\)?\w*)=(-?[0-9]\d*\.\d*)',line)
                print(t_x, len(t_x))
                #print(t_x)
                for i in range(len(t_x)):
                    if t_x[i][0]=='time':
                        pass
                    else:
                        print(x.shape, t, i)
                        x[t,i]=t_x[i][1] #由于index填入模型名称的顺序就是index列表的序号，因此行与list一一对应
                #print(x)
    #生成DataFrame
    df.append(DataFrame(x,columns=col,index=dex))
    col=[]   
    dex=[]
    temp_list=[]              #当前数据集frame生成完毕，临时列表清零
    print('DateFrame accomplished, input df[',d,']to get df_',data[d])
#df


0
['ckpt/west/srnn/srnn_model.D=1/2020-12-03_02-48-55', 'ckpt/west/srnn/srnn_model.D=5/2020-12-03_08-34-29', 'ckpt/west/srnn/srnn_model.D=3/2020-12-03_06-32-27', 'ckpt/west/best/2020-12-03_07-20-37', 'ckpt/west/vaecl/vaecl_model.D=5/2020-12-01_14-36-25', 'ckpt/west/vaecl/vaecl_model.D=5/2020-12-01_14-56-24', 'ckpt/west/vaecl/vaecl_model.D=3/2020-12-01_13-35-32', 'ckpt/west/vaecl/vaecl_model.D=3/2020-12-01_13-44-08', 'ckpt/west/vaecl/vaecl_model.D=1/2020-12-01_12-59-38', 'ckpt/west/vaecl/vaecl_model.D=1/2020-12-01_12-52-36', 'ckpt/west/vrnn/vrnn_model.D=1/2020-12-03_03-43-54', 'ckpt/west/vrnn/vrnn_model.D=5/2020-12-03_11-40-14', 'ckpt/west/vrnn/vrnn_model.D=3/2020-12-03_07-20-37']
[('likelihood', '-195.2691'), ('ob_rrse', '0.1745'), ('ob_Pressure_pear', '0.9836'), ('pred_rrse', '0.9486'), ('pred_Pressure_pear', '0.9287'), ('time', '3.8245')] 6
(13, 5) 0 0
(13, 5) 0 1
(13, 5) 0 2
(13, 5) 0 3
(13, 5) 0 4
[('likelihood', '250.1230'), ('ob_rrse', '1.3385'), ('ob_Pressure_pear', '0.1191'), (

In [14]:
print(data[0])
df[0].to_csv('df_%s.csv' % data[0])
df[0]

west


,likelihood,ob_rrse,ob_Pressure_pear,pred_rrse,pred_Pressure_pear
srnn_model.D=1//2020-12-03_02-48-55,-195.2691,0.1745,0.9836,0.9486,0.9287
srnn_model.D=5//2020-12-03_08-34-29,250.1230,1.3385,0.1191,5.3798,0.5255
srnn_model.D=3//2020-12-03_06-32-27,251.4995,1.3410,0.1187,5.0190,0.4848
best//2020-12-03_07-20-37,-269.8546,0.1650,0.9845,0.6870,0.9374
vaecl_model.D=5//2020-12-01_14-36-25,184.0225,0.8682,0.5637,1.3609,0.5782
vaecl_model.D=5//2020-12-01_14-56-24,262.9291,1.3140,0.1370,3.2397,0.2815
vaecl_model.D=3//2020-12-01_13-35-32,205.2291,0.9058,0.5261,1.4783,0.5473
vaecl_model.D=3//2020-12-01_13-44-08,195.4979,0.8732,0.5577,1.3655,0.5879
vaecl_model.D=1//2020-12-01_12-59-38,172.1628,0.8014,0.6228,1.3066,0.6332
vaecl_model.D=1//2020-12-01_12-52-36,260.3790,1.1747,0.2904,2.2883,0.3535


In [15]:
print(data[1])
df[1].to_csv('df_%s.csv' % data[1])
df[1]

west_con


,likelihood,ob_rrse,ob_UC_pear,ob_Pressure_pear,pred_rrse,pred_UC_pear,pred_Pressure_pear
srnn_model.D=1//2020-12-24_15-15-55,598.4109,1.4451,-0.0214,-0.0130,12.9590,-0.1381,-0.1545
vrnn_model.D=1//2020-12-24_14-51-13,459.0544,1.2943,0.0504,0.2443,4.5629,-0.0979,-0.0353
srnn_model.D=3//2020-12-24_15-28-49,613.9923,1.4664,-0.0131,0.0471,10.3725,-0.0955,0.3854
vaecl_model.D=3//2020-12-24_15-48-47,643.6994,1.4718,0.0107,0.0001,11.7608,-0.0439,-0.1408
vrnn_model.D=3//2020-12-24_15-03-24,555.8284,1.4779,0.0380,0.0899,7.2874,-0.1113,-0.2555
vaecl_model.D=1//2020-12-24_15-42-02,653.5730,1.5060,0.0300,0.0090,7.3226,-0.0365,0.1370


In [16]:
print(data[2])
df[2].to_csv('df_%s.csv' % data[2])
df[2]

winding


,likelihood,ob_rrse,ob_tension1_pear,ob_tension2_pear,pred_rrse,pred_tension1_pear,pred_tension2_pear
srnn_model.D=1//2020-12-02_22-35-24,310.0891,0.7333,0.8327,0.6287,0.5818,0.9560,0.7904
srnn_model.D=5//2020-12-03_01-23-11,251.7014,0.7026,0.8465,0.6684,0.5479,0.9549,0.8136
srnn_model.D=3//2020-12-02_23-58-35,236.3744,0.6844,0.8743,0.6737,0.5660,0.9461,0.8111
vaecl_model.D=5//2020-12-01_12-06-05,489.6334,1.1418,0.4538,0.2758,1.5354,0.6327,0.4230
vaecl_model.D=5//2020-12-01_12-07-01,583.4399,1.4375,0.0211,-0.0012,19.9477,0.1552,0.0305
vaecl_model.D=3//2020-12-01_11-15-02,479.3863,1.0689,0.4676,0.3685,1.3256,0.6372,0.5232
vaecl_model.D=3//2020-12-01_11-15-52,569.9989,1.4019,0.0633,0.0725,5.3267,0.3056,0.2313
vaecl_model.D=1//2020-12-01_10-43-25,499.0857,1.1532,0.4676,0.2001,1.6563,0.6518,0.3735
vaecl_model.D=1//2020-12-01_10-43-27,525.5965,1.2391,0.3308,0.1623,1.9403,0.5767,0.3272
vrnn_model.D=1//2020-12-02_19-30-27,100.7653,0.3344,0.9872,0.8845,0.6200,0.9061,0.7935


In [17]:
print(data[3])
df[3].to_csv('df_%s.csv' % data[3])
df[3]

cstr


,likelihood,ob_rrse,ob_concentration_pear,ob_temp(K)_pear,pred_rrse,pred_concentration_pear,pred_temp(K)_pear
srnn_model.D=1//2020-12-02_13-04-20,-247.0229,0.3162,0.9399,0.9496,0.2278,0.9745,0.9791
srnn_model.D=5//2020-12-02_19-07-53,-273.7450,0.3155,0.9387,0.9523,0.2267,0.9758,0.9813
srnn_model.D=3//2020-12-02_15-44-21,-253.7365,0.3223,0.9341,0.9498,0.2339,0.9747,0.9792
vaecl_model.D=5//2020-12-01_09-30-04,-353.8404,0.1585,0.9846,0.9885,0.2335,0.9729,0.9777
vaecl_model.D=5//2020-12-01_08-43-09,-374.1149,0.1472,0.9867,0.9895,0.2297,0.9741,0.9779
vaecl_model.D=3//2020-12-01_07-40-36,-403.4965,0.1427,0.9876,0.9909,0.2239,0.9760,0.9792
vaecl_model.D=3//2020-12-01_07-41-34,-418.2319,0.1439,0.9881,0.9905,0.2196,0.9780,0.9802
vaecl_model.D=1//2020-12-01_07-13-54,-413.2803,0.1293,0.9912,0.9922,0.2469,0.9697,0.9731
vaecl_model.D=1//2020-12-01_07-13-00,-348.1160,0.1735,0.9823,0.9852,0.2826,0.9622,0.9664
vrnn_model.D=1//2020-12-01_07-09-41,-154.2750,0.4765,0.8952,0.8827,0.4667,0.9608,0.9671
